In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
csv_filename = "log_20161110.csv"

+ csv ファイルは、列ヘッダなし・列数8列・全5902行
+ 各列は以下の通り：
    + ページ（パス、文字列）
    + ページビュー数（整数）
    + ページ別訪問数（整数）
    + 平均ページ滞在時間（整数）
    + 閲覧開始数（整数）
    + 直帰率（実数）
    + 離脱率（実数）
    + コンテンツサイズ（整数）

In [ ]:
# np_logcsv = np.loadtxt("log_20161110.csv", delimiter=",")
# # @> ValueError（最初の列が文字列なので実数値にparseできなくて怒られる＞＜）

In [ ]:
filename_queue = tf.train.string_input_producer([csv_filename])

In [ ]:
# TextLineReader 生成（1行ずつ読み込む Reader）
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [ ]:
# CSVデコード
# ※ 列は8列、文字列、整数、整数、整数、整数、実数、実数、整数。だが処理の都合上、
# 　 最初以外は実数として扱う↓）
page, pv, visits, duration, entrances, bounce_rate, exit_rate, contents_size = (
    tf.decode_csv(value, [['path'], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0]]))
x = tf.pack([pv, visits, duration, entrances, bounce_rate, contents_size])
y = exit_rate

In [ ]:
X_batch, Y_batch = tf.train.batch(
      [x, y], batch_size=128)

In [ ]:
# セッション開始
with tf.Session() as sess:
    # 入力 enqueue スレッド開始
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(
                    sess=sess, coord=coord)

    try:
        # ミニバッチ処理ループ
        while not coord.should_stop():
            XData, YData = sess.run([X_batch, Y_batch])
            # 取り敢えず内容確認して break
            print((XData, YData))
            break

    finally:
        coord.request_stop()
        coord.join(threads)

In [ ]:
csv_lines = open(csv_filename, "r").readlines()

In [ ]:
len(csv_lines)

In [ ]:
csv_lines[:6]

In [ ]:
train_idxs = np.random.choice(range(5902), 5902*5//6, replace=False)

In [ ]:
train_data = np.array(csv_lines)[train_idxs]

In [ ]:
csv_data = np.array(csv_lines)
np.random.shuffle(csv_data)
csv_data[:6]

In [ ]:
train_num = len(csv_lines) * 5 // 6
train_data = csv_data[:train_num]
test_data = csv_data[train_num:]

(len(train_data), len(test_data))

In [ ]:
with open("train.csv", "w") as f:
    for line in train_data:
        f.write(line)

In [ ]:
with open("test.csv", "w") as f:
    for line in test_data:
        f.write(line)

In [ ]:
def read_data(filename):
    filename_queue = tf.train.string_input_producer([filename])

    # TextLineReader 生成（1行ずつ読み込む Reader）
    reader = tf.TextLineReader()
    key, value = reader.read(filename_queue)

    # CSVデコード
    # ※ 列は8列、文字列、整数、整数、整数、整数、実数、実数、整数。だが処理の都合上、
    # 　 最初以外は実数として扱う↓）
    page, pv, visits, duration, entrances, bounce_rate, exit_rate, contents_size = (
        tf.decode_csv(value, [['path'], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0]]))
    x = tf.pack([pv, visits, duration, entrances, bounce_rate, contents_size])
    # y = exit_rate
    # # round 
    # y = tf.round(exit_rate)
    # # threshold 0.7 
    # y = tf.floor(exit_rate + 0.3)
    # threshold 0.4 
    y = tf.floor(exit_rate + 0.6)

    x_, y_ = tf.train.batch(
          [x, y], batch_size=128)
    return x_, tf.reshape(y_, [-1])

In [ ]:
x_train, y_train = read_data("train.csv")

In [ ]:
(x_train, y_train)

In [ ]:
x_test, y_test = read_data("test.csv")

In [ ]:
# W = tf.Variable(tf.zeros([6, 1]))
W1 = tf.Variable(tf.truncated_normal([6, 10], stddev=2.0/16))
b1 = tf.Variable(tf.zeros([10]))
W2 = tf.Variable(tf.truncated_normal([10, 4], stddev=2.0/14))
b2 = tf.Variable(tf.zeros([4]))
W3 = tf.Variable(tf.truncated_normal([4, 1], stddev=2.0/5))
b3 = tf.Variable(tf.zeros([1]))

In [ ]:
def inference(x, W1=W1, b1=b1, W2=W2, b2=b2, W3=W3, b3=b3):
    a1 = tf.matmul(x, W1) + b1
    z1 = tf.nn.relu(a1)
    a2 = tf.matmul(z1, W2) + b2
    z2 = tf.nn.relu(a2)
    a3 = tf.matmul(z2, W3) + b3
    # return tf.nn.sigmoid(tf.reshape(a3, [-1]))
    return tf.reshape(a3, [-1])

In [ ]:
y = inference(x_train)

In [ ]:
y

In [ ]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(y, y_train))
# loss = tf.reduce_mean(tf.square(y_train - y))

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
y_eval = tf.nn.sigmoid(inference(x_test))
# y_eval = inference(x_test)

In [ ]:
init = tf.initialize_all_variables()

In [ ]:
sess = tf.InteractiveSession()
sess.run(init)

In [ ]:
# 入力 enqueue スレッド開始
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(
                sess=sess, coord=coord)

In [ ]:
try:
    # ミニバッチ処理ループ
    cnt = 0
    while not coord.should_stop() and cnt < 3000:
        _, loss_ = sess.run([train_step, loss])
        if cnt % 100 == 0:
            print(loss_)
            y_test_, y_eval_ = sess.run([y_test, y_eval])
            print(np.mean(y_test_ == np.round(y_eval_)))
            # print(np.mean((y_eval_ - y_test_)**2))
        cnt += 1
        # break
    W1_, b1_, W2_, b2_, W3_, b3_ = sess.run([W1, b1, W2, b2, W3, b3])
finally:
    coord.request_stop()
    coord.join(threads)

In [ ]:
sess.close()

In [ ]:
loss_

In [ ]:
y_test_

In [ ]:
(y_eval_, np.round(y_eval_))

In [ ]:
(W1_, b1_)